# 1) Setup

In [2]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import math
import scipy as sp
from scipy import stats
from copy import copy
import time

import pymongo
from pymongo import MongoClient
from datetime import datetime
from datetime import timedelta
print("done")

done


In [59]:
#import functions from python files

#1) data import & filtering
from get_data import *
from get_adjacent import *
from get_valid1 import *
from get_times import *
from get_congestion import *
from get_valid2 import *
from get_length import *

from select_data import *
from select_data_edge import *

from underscore_prefix import *
from underscore_suffix import *

print("done")

done


# 2) AAF Y4

In [73]:
#1) Import data from PyMongo
df = get_data(address = "aaf_y4")
df = df[df["edge_id"] != "WayPoint28_Feuerloescher"]
adjacent = get_adjacent("aaf_map.yaml")
nodes = list(adjacent.keys())
length = get_length("aaf_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))
df.head(2)

Time taken (get_adjacent): 0.5959110260009766 secs
Time taken (get_length): 0.5898942947387695 secs
23044
Time taken (get_adjacent): 0.6024034023284912 secs
23044


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node
0,583b30dfb63c4d6eb86ff88e,success,Station,ChargingPoint,Station_ChargingPoint,"Sunday, November 27 2016, at 20:15:43 hours","Sunday, November 27 2016, at 20:15:43 hours","Sunday, November 27 2016, at 20:15:24 hours","{'inserted_at': 2016-11-27 19:15:43.999000, 's...",0.624420,19.893930,ChargingPoint
1,583b31d7b63c4d6eb86ff88f,success,ChargingPoint,Station,ChargingPoint_Station,"Sunday, November 27 2016, at 20:19:51 hours","Sunday, November 27 2016, at 20:19:34 hours","Sunday, November 27 2016, at 20:19:27 hours","{'inserted_at': 2016-11-27 19:19:51.098000, 's...",17.028488,23.572407,Station


In [74]:
#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)
print("done")

Time taken (get_times): 20.0078706741333 secs
done


In [75]:
#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))
df.head(2)

0 iterations in 0.0 seconds
5000 iterations in 25.27942705154419 seconds
10000 iterations in 56.276663064956665 seconds
15000 iterations in 90.0684084892273 seconds
20000 iterations in 121.18451523780823 seconds
Time taken (get_congestion): 139.92115569114685 seconds
Unimodal:  22581
Multimodal:  463


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
0,583b30dfb63c4d6eb86ff88e,success,Station,ChargingPoint,Station_ChargingPoint,2016-11-27 20:15:43,2016-11-27 20:15:43,2016-11-27 20:15:24,2016-11-27 19:15:43.999000,0.624420,19.893930,ChargingPoint,1
1,583b31d7b63c4d6eb86ff88f,success,ChargingPoint,Station,ChargingPoint_Station,2016-11-27 20:19:51,2016-11-27 20:19:34,2016-11-27 20:19:27,2016-11-27 19:19:51.098000,17.028488,23.572407,Station,1


In [76]:
#5) remove final, unsuccessful, multimodal runs
df_aaf = get_valid2(df)
df_aaf.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station,ChargingPoint,Station_ChargingPoint,0.624420,19.893930
1,ChargingPoint,Station,ChargingPoint_Station,17.028488,23.572407


In [77]:
#6) save to excel
df_aaf.to_excel("aaf_y4.xlsx")
print("done")

done


In [78]:
# retrieve data
df_aaf = pd.read_excel("aaf_y4.xlsx")
df_aaf = df_aaf[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_aaf.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station,ChargingPoint,Station_ChargingPoint,0.624420,19.893930
1,ChargingPoint,Station,ChargingPoint_Station,17.028488,23.572407


# 3) AAF Y3

In [70]:
#1) Import data from PyMongo
df = get_data(address = "aaf_y3")
df = df[df["edge_id"] != "WayPoint28_Feuerloescher"]
adjacent = get_adjacent("aaf_map.yaml")
nodes = list(adjacent.keys())
length = get_length("aaf_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
#adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))

#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)

#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))

#5) remove final, unsuccessful, multimodal runs
df_aaf = get_valid2(df)
df_aaf.head(2)

Time taken (get_adjacent): 0.5860164165496826 secs
Time taken (get_length): 0.6682631969451904 secs
18142
18142
Time taken (get_times): 11.171806335449219 secs
0 iterations in 0.0 seconds
5000 iterations in 20.611332416534424 seconds
10000 iterations in 41.289241313934326 seconds
15000 iterations in 62.16041111946106 seconds
Time taken (get_congestion): 75.2370548248291 seconds
Unimodal:  17867
Multimodal:  275


,origin,target,edge_id,time_to_waypoint,operation_time
1,Station,WayPoint91,Station_WayPoint91,0.071773,4.097293
2,WayPoint91,WayPoint8,WayPoint91_WayPoint8,5.784959,13.085477


In [71]:
#6) save to excel
df_aaf.to_excel("aaf_y3.xlsx")
print("done")

done


In [72]:
# retrieve data
df_aaf = pd.read_excel("aaf_y3.xlsx")
df_aaf = df_aaf[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_aaf.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station,WayPoint91,Station_WayPoint91,0.071773,4.097293
1,WayPoint91,WayPoint8,WayPoint91_WayPoint8,5.784959,13.085477


# 4) AAF Y2

In [67]:
#1) Import data from PyMongo
df = get_data(address = "aaf_y2")
df = df[df["edge_id"] != "Wp87_WP2"]
df = df[df["edge_id"] != "WayPoint28_Feuerloescher"]

adjacent = get_adjacent("aaf_map.yaml")
nodes = list(adjacent.keys())
length = get_length("aaf_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
#adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))

#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)

#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))

#5) remove final, unsuccessful, multimodal runs
df_aaf = get_valid2(df)
df_aaf.head(2)

Time taken (get_adjacent): 0.5645287036895752 secs
Time taken (get_length): 0.6239008903503418 secs
5446
5446
Time taken (get_times): 2.0456933975219727 secs
0 iterations in 0.0 seconds
5000 iterations in 9.356168270111084 seconds
Time taken (get_congestion): 10.188511848449707 seconds
Unimodal:  5409
Multimodal:  37


,origin,target,edge_id,time_to_waypoint,operation_time
1,Station,WayPoint2,Station_WayPoint2,0.328289,2.676215
2,WayPoint2,WayPoint86,WayPoint2_WayPoint86,0.236417,9.111409


In [68]:
#6) save to excel
df_aaf.to_excel("aaf_y2.xlsx")
print("done")

done


In [69]:
# retrieve data
df_aaf = pd.read_excel("aaf_y2.xlsx")
df_aaf = df_aaf[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_aaf.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station,WayPoint2,Station_WayPoint2,0.328289,2.676215
1,WayPoint2,WayPoint86,WayPoint2_WayPoint86,0.236417,9.111409


# 5) TSC y3

In [82]:
#1) Import data from PyMongo
df = get_data(address = "tsc_y3")

adjacent = get_adjacent("tsc_map.yaml")
nodes = list(adjacent.keys())
length = get_length("tsc_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
#adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))

#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)

#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))

#5) remove final, unsuccessful, multimodal runs
df_tsc = get_valid2(df)
df_tsc.head(2)

Time taken (get_adjacent): 0.6584620475769043 secs
Time taken (get_length): 0.735328197479248 secs
6828
6828
Time taken (get_times): 3.2972218990325928 secs
0 iterations in 0.0 seconds
5000 iterations in 11.889139890670776 seconds
Time taken (get_congestion): 16.39375376701355 seconds
Unimodal:  6397
Multimodal:  431


,origin,target,edge_id,time_to_waypoint,operation_time
0,ChargingPoint,Station,ChargingPoint_Station,26.468006,38.898573
2,Station,ChargingPoint,Station_ChargingPoint,0.413475,68.313145


In [83]:
#6) save to excel
df_tsc.to_excel("tsc_y3.xlsx")
print("done")

done


In [84]:
# retrieve data
df_tsc = pd.read_excel("tsc_y3.xlsx")
df_tsc = df_tsc[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_tsc.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,ChargingPoint,Station,ChargingPoint_Station,26.468006,38.898573
1,Station,ChargingPoint,Station_ChargingPoint,0.413475,68.313145


# 6) TSC y4 - Betty

In [85]:
#1) Import data from PyMongo
df = get_data(address = "tsc_betty")

adjacent = get_adjacent("tsc_map.yaml")
nodes = list(adjacent.keys())
length = get_length("tsc_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
#adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))

#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)

#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))

#5) remove final, unsuccessful, multimodal runs
df_tsc = get_valid2(df)
df_tsc.head(2)

Time taken (get_adjacent): 0.5814762115478516 secs
Time taken (get_length): 0.63150954246521 secs
3328
3328
Time taken (get_times): 1.1859149932861328 secs
0 iterations in 0.0 seconds
Time taken (get_congestion): 5.6266255378723145 seconds
Unimodal:  3323
Multimodal:  5


,origin,target,edge_id,time_to_waypoint,operation_time
0,Station1,SmallKitchen,Station1_SmallKitchen,0.047758,9.263602
1,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.088824,8.628814


In [86]:
#6) save to excel
df_tsc.to_excel("tsc_betty.xlsx")
print("done")

done


In [87]:
# retrieve data
df_tsc = pd.read_excel("tsc_betty.xlsx")
df_tsc = df_tsc[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_tsc.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station1,SmallKitchen,Station1_SmallKitchen,0.047758,9.263602
1,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.088824,8.628814


# 7) TSC y4 - Bob

In [88]:
#1) Import data from PyMongo
df = get_data(address = "tsc_bob")

adjacent = get_adjacent("tsc_map.yaml")
nodes = list(adjacent.keys())
length = get_length("tsc_map.yaml")
edges = list(length.keys())
df = df.loc[df["origin"].isin(nodes)]
df = df.loc[df["target"].isin(nodes)]
df = df.loc[df["edge_id"].isin(edges)]
print(len(df))

#2) Remove data with invalid origin/target
#adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))

#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)

#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))

#5) remove final, unsuccessful, multimodal runs
df_tsc = get_valid2(df)
df_tsc.head(2)

Time taken (get_adjacent): 0.5899572372436523 secs
Time taken (get_length): 0.6293232440948486 secs
9012
9012
Time taken (get_times): 4.191533327102661 secs
0 iterations in 0.0009975433349609375 seconds
5000 iterations in 12.459723711013794 seconds
Time taken (get_congestion): 22.725107431411743 seconds
Unimodal:  8986
Multimodal:  26


,origin,target,edge_id,time_to_waypoint,operation_time
1,Station1,SmallKitchen,Station1_SmallKitchen,0.054096,4.340037
2,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.077031,5.906566


In [89]:
#6) save to excel
df_tsc.to_excel("tsc_bob.xlsx")
print("done")

done


In [91]:
# retrieve data
df_tsc_bob = pd.read_excel("tsc_bob.xlsx")
df_tsc_bob = df_tsc_bob[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_tsc_bob.head(2)

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station1,SmallKitchen,Station1_SmallKitchen,0.054096,4.340037
1,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.077031,5.906566
